In [ ]:
# @title 🚀 INICIAR ESTAÇÃO VISUAL (LIMPEZA + MOTOR + TÚNEL)
import os, subprocess, time, re, socket

def is_port_open(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

print("🧹 1. Limpando processos antigos...")
!pkill -9 -f engine.py
!pkill -9 cloudflared
!rm -f engine.py*

print("📦 2. Instalando dependências...")
!pip install -q diffusers transformers accelerate flask flask-cors

print("📥 3. Baixando motor atualizado...")
!wget -q https://raw.githubusercontent.com/Miraplay2025/Gerador-de-imagem/main/engine.py -O engine.py

print("⚙️ 4. Iniciando Servidor Flask...")
subprocess.Popen(["python3", "engine.py"])

print("⏳ 5. Aguardando a IA carregar (isso pode levar 30s)...")
while not is_port_open(5000):
    time.sleep(2)
    print(".", end="")

print("\n✅ Servidor detectado na porta 5000!")

print("🌐 6. Ativando Túnel Cloudflare...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

p = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:5000'], 
                     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in p.stdout:
    if ".trycloudflare.com" in line:
        url = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", line)
        if url:
            print("\n" + "="*60)
            print(f"🚀 SISTEMA ONLINE: {url.group(0)}")
            print("="*60)
            print("MANTENHA ESTA ABA RODOANDO!")
            break

# Mantém o processo vivo
try:
    while True:
        time.sleep(100)
except KeyboardInterrupt:
    print("Encerrado.")